In [1]:
import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session



sess = Session()

role = get_execution_role()
model_dir = "/home/ec2-user/SageMaker/smoke-detection-smokeynet/src" # Replace with S3

In [2]:
pt_mnist_model_data = sess.upload_data(
    path="/home/ec2-user/SageMaker/smoke-detection-smokeynet/src/last.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
)

In [3]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir=model_dir,
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.5.0",
    py_version="py3",
)

#### Deploy Model

In [4]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.g4dn.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

----------!

In [5]:
import random
import numpy as np
import boto3
from io import BytesIO
import pickle
from urllib.parse import urlparse

dummy_data = np.random.rand(2, 45, 2, 3, 224, 224)
s3_client = boto3.client("s3")
# bucket = "smokynet-inference-images-processed"
# bytes_ = BytesIO()
# np.save(bytes_, dummy_data, allow_pickle=True)
# s3_uri = "s3://smokynet-inference-images-processed/test/dummy_data"
# bytes_.seek(0)
# parsed_s3 = urlparse(s3_uri)
# s3.upload_fileobj(
#         Fileobj=bytes_, Bucket=bucket, Key=parsed_s3.path[1:])
my_array_data = BytesIO()
pickle.dump(dummy_data, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'smokynet-inference-images-processed', 'processed_data.pkl')

In [ ]:
arr = np.random.rand(1, 1, 2, 3, 224, 224)
arr.size * arr.itemsize

In [6]:


res = predictor.predict({'bucket_name' : 'smokynet-inference-images-processed'})

In [7]:
res

[[1.6568760871887207], [1.6596603393554688]]

In [ ]:
!pip install transformers

#### Delete endpoint

In [ ]:
import os

if not local_mode:
    predictor.delete_endpoint()
else:
    os.system("docker container ls | grep 8080 | awk '{print $1}' | xargs docker container rm -f")

In [ ]:
import boto3
import numpy as np
from io import BytesIO
from urllib.parse import urlparse

s3_uri = "s3://smokynet-inference-images-processed/test/dummy_data"
bucket_name = "smokynet-inference-images-processed"
bytes_ = BytesIO()
parsed_s3 = urlparse(s3_uri)
s3.download_fileobj(
    Fileobj=bytes_, Bucket=bucket_name, Key='test/dummy_data'
)
bytes_.seek(0)
data = np.load(bytes_, allow_pickle=True)


In [ ]:
s3_uri = "s3://smokynet-inference-images-processed/test/dummy_data"
bytes_ = BytesIO()
parsed_s3 = urlparse(s3_uri)
s3.download_fileobj(
    Fileobj=bytes_, Bucket=parsed_s3.netloc, Key=parsed_s3.path[1:]
)
bytes_.seek(0)
data=  np.load(bytes_, allow_pickle=True)

In [ ]:
type(data)

In [ ]:
import boto3
import io
import pickle

s3_client = boto3.client('s3')

my_array = np.random.rand(2, 45, 2, 3, 224, 224)

# upload without using disk
my_array_data = io.BytesIO()
pickle.dump(my_array, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'smokynet-inference-images-processed', 'your-file.pkl')

# download without using disk
my_array_data2 = io.BytesIO()
s3_client.download_fileobj('smokynet-inference-images-processed', 'your-file.pkl', my_array_data2)
my_array_data2.seek(0)
my_array2 = pickle.load(my_array_data2)

# check that everything is correct
np.allclose(my_array, my_array2)

In [ ]:
my_array2.shape

In [ ]:
import torch
device='cpu'
data = torch.tensor(my_array2, dtype=torch.float32, device=device)

In [ ]:
data = torch.tensor(my_array2, dtype=torch.float32, device=device)

In [ ]:
data.shape

In [ ]:

import json
import logging
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
# from main_model import MainModel
# import util_fns
# from lightning_module import LightningModule
import boto3
import numpy as np
from io import BytesIO
from urllib.parse import urlparse
import pickle

def input_fn(request_body, request_content_type):
    s3_client = boto3.client("s3")
    #assert request_content_type == "application/json"
#     data = json.loads(request_body)["inputs"]
#     data = torch.tensor(data, dtype=torch.float32, device=device)
#     s3_uri = "s3://smokynet-inference-images-processed/test/dummy_data"
#     bucket_name = json.loads(request_body)["bucket_name"]
#     bytes_ = BytesIO()
#     parsed_s3 = urlparse(s3_uri)
#     s3.download_fileobj(
#         Fileobj=bytes_, Bucket=bucket_name, Key=parsed_s3.path[1:]
#     )
#     bytes_.seek(0)
#     data = np.load(bytes_, allow_pickle=True)
    my_array_data2 = BytesIO()
    s3_client.download_fileobj('smokynet-inference-images-processed', 'processed_data.pkl', my_array_data2)
    my_array_data2.seek(0)
    my_array2 = pickle.load(my_array_data2)
    data = torch.tensor(my_array2, dtype=torch.float32, device=device)
    return data

In [ ]:
input_fn('sss','s')